In [ ]:
import pickle
import cv2
from glob import glob 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pipeline as pl

In [ ]:
path = {'calibration': './camera_cal/',
        'example': './examples/',
        'test': './test_images/'}
files  = {key:[] for key in path}
images = {key:[] for key in path}

In [ ]:
files['calibration'] = glob(path['calibration'] + 'calibration*.jpg')
files['example']     = glob(path['example'] + '*.jpg')
files['test']        = glob(path['test'] + '*.jpg')

for key in files:
    print("Load {} images...".format(key))
    for file in files[key]:
        images[key].append(mpimg.imread(file))
        #print('{}'.format(file))
    #print('\n')

In [ ]:
fig, axes = plt.subplots(3,1, figsize=(18,18))
for ax, key in zip(axes.flatten(), images):
    ax.imshow(images[key][0])
plt.show()

In [ ]:
for key in images:
    for img in images[key][0:3]:
        plt.imshow(img)
        plt.show()

## Calibrate Camera

In [ ]:
imgpoints = [] # 2D
objpoints = [] # 3D
nx = 9
ny = 6
objp = np.zeros((nx*ny, 3), np.float32)
objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)

distortion_coefficients = []
camera_matrices = []

def cal_undistort(image, objpoints, imgpoints, nx, ny, distortion_coefficients, camera_matrices):
    # Use cv2.calibrateCamera() and cv2.undistort()
    img = image.copy()
    
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # Finding chessboard corners (for an 8x6 board):
    ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)
    print("Found corners: {0}".format(ret))
    
    if ret==True:
        imgpoints.append(corners)
        objpoints.append(objp)

        # Drawing detected corners on an image:
        img = cv2.drawChessboardCorners(img, (nx,ny), corners, ret)

        # Camera calibration, given object points, image points, and the shape of the grayscale image:
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

        # Undistorting a test image:
        dst = cv2.undistort(img, mtx, dist, None, mtx)
        
        camera_matrices.append(mtx)
        distortion_coefficients.append(dist)
    else:
        dst = np.zeros_like(img)
        
        
        
    return img, dst

for img in images['calibration'][0::]:
    
    original, undistorted = cal_undistort(img, objpoints, imgpoints, nx, ny, distortion_coefficients, camera_matrices) 
    
# Get the last camera matrix and distortion coefficients as the final calibration set
distortion_coefficients = np.array(distortion_coefficients).squeeze()
camera_matrices = np.array(camera_matrices).squeeze()

mtx = camera_matrices[-1,::]
dist = distortion_coefficients[-1,::]

singuar_values = []
for cm in camera_matrices:
    U, s, V = np.linalg.svd(cm, full_matrices=True)
    singuar_values.append(s)
singuar_values = np.array(singuar_values)

### Analyse Calibration Results

In [ ]:
fig, axes = plt.subplots(3,1, figsize=(18,10))
fig.subplots_adjust(hspace=0.15, wspace=0.15)
axes[0].plot(distortion_coefficients)
axes[0].set_title("Distortion Coefficient")
axes[0].set_ylabel("Value of Distortion Coefficient")
axes[0].grid('on')

axes[1].plot(camera_matrices.reshape(-1, 9))
axes[1].set_title("Camera Matrix Elements")
axes[1].set_ylabel("Value of Camera Matrix Element")
axes[1].grid('on')

axes[2].plot(singuar_values[:,0])
axes[2].set_title("First Singular Value of Camera Matrix")
axes[2].set_ylabel("First Singl. Value of Cam. Matrix")
axes[2].grid('on')

plt.show()

### Compare Original and Undistorted Images

In [ ]:
nb_images = 3

fig, axes = plt.subplots(nb_images, 2, figsize=(18, (nb_images * 5)))
fig.subplots_adjust(hspace=0.15, wspace=0.15)
for i, img in enumerate(images['calibration'][0:nb_images]):
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    axes[i,0].imshow(img)
    axes[i,1].imshow(dst)
axes[0,0].set_title('Original Images', fontsize=26)
axes[0,1].set_title('Undistorted Images', fontsize=26)
plt.show()

## Compute Perspective Transform

In [ ]:
img = images['test'][0]

dst = cv2.undistort(img, mtx, dist, None, mtx)

y1, y2 = 720.0, 550.0
x1, x2 = 220.0, 1100.0
dx = 240.0
dy = -25
src_rect = np.zeros((4, 2), dtype = "float32")
src_rect[0,:] = [x2-dx-20,y2]
src_rect[1,:] = [x2,y1]
src_rect[2,:] = [x1,y1]
src_rect[3,:] = [x1+dx,y2]

dst_rect = np.zeros((4, 2), dtype = "float32")
dst_rect[0,:] = [x2,y2-dy]
dst_rect[1,:] = [x2,y1]
dst_rect[2,:] = [x1,y1]
dst_rect[3,:] = [x1,y2-dy]

for c in src_rect:
    result = cv2.circle(img.copy(), tuple(c), 1, 0xFF, 4)
    
for d in dst_rect:
    result = cv2.circle(img.copy(), tuple(d), 1, 0x00, 10)
    

## Undistort Camera Images

### Comparison of Original and Calibrated Image

In [ ]:
alpha = 0.5
overlay = dst.copy()
output = img.copy()
# Apply the overlay
result = cv2.addWeighted(overlay, alpha, output, 1 - alpha, 0, output)

fig, axes = plt.subplots(2,2, figsize=(18,11))
fig.subplots_adjust(hspace=0.15, wspace=0.25)

fontsize = 22

axes[0,0].imshow(img)
axes[0,0].set_title('Original Image', fontsize=fontsize)
axes[0,1].imshow(dst)
axes[0,1].set_title('Undistorted Image', fontsize=fontsize)
axes[1,0].imshow(result)
axes[1,0].set_title('Overlay of the Original and the Undistorted Image', fontsize=fontsize)
axes[1,1].imshow(img-dst)
axes[1,1].set_title('Difference Image: Original-Undistorted', fontsize=fontsize)

plt.show()

In [ ]:
M = cv2.getPerspectiveTransform(src_rect, dst_rect)

# Compute the inverse perspective transform:
Minv = cv2.getPerspectiveTransform(dst_rect, src_rect)

# Warp an image using the perspective transform, M:
warped = cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]), flags=cv2.INTER_LINEAR)
gray   = cv2.cvtColor(warped, cv2.COLOR_RGB2GRAY)
plt.figure(figsize=(20,20))
plt.imshow(warped)
plt.show()

In [ ]:
result, _ = pl.random_affine_transformation(img, 0, shear_range=600)
fig = plt.figure(figsize=(18,10))
plt.imshow(result)
plt.show()